<div style="font-family: Arial, Helvetica, sans-serif;">
    <div style="display: flex;padding-top: 20px">
        <div><strong>Môn học:</strong> Trực quan hóa dữ liệu</div>
    </div>
    <div style="display: flex;padding-top: 20px">
        <div style="padding-right: 10px;"><strong>Lớp:</strong> 22KHDL</div>
        <div></div>
    </div>
    <div>
        <div style="display: flex;padding-top: 20px">
            <div style="padding-right: 10px;"><strong>Thành viên:</strong></div>
            <div></div>
        </div>
        <table style="font-size: 15px; display:flex;padding-top: 20px">
            <tr>
                <th>No.</th>
                <th>Student ID</th>
                <th>Name</th>
            </tr>
            <tr>
                <td>1</td>
                <td>22127008</td>
                <td style="text-align:left;">Đặng Châu Anh</td>
            </tr>
            <tr>
                <td>2</td>
                <td>22127014</td>
                <td style="text-align:left;">Nguyễn Kim Anh</td>
            </tr>
            <tr>
                <td>4</td>
                <td>22127170</td>
                <td style="text-align:left;">Trần Dịu Huyền</td>
            </tr>
        </table>
    </div>
    <div style="font-size: 25px ;font-weight: 800; text-align: center;padding-top: 20px;">LAB 01</div>
    <div style="font-size: 20px ;font-weight: 800; text-align: center;padding-top: 20px;">THU THẬP DỮ LIỆU VÀ TRỰC QUAN HÓA DỮ LIỆU</div>
</div>

# **Mục lục**

- [**Giới thiệu**](#intro)
    - [Mục tiêu đồ án](#objectives)
    - [Thông tin dữ liệu](#data)
- [**Thư viện sử dụng**](#packages)
- [**Thu thập dữ liệu**](#collect)


<a id="intro"></a>
# **I. Giới thiệu**
<a id="objectives"></a>
## **1. Mục tiêu đồ án:**

<a id="data"></a>
## **2. Thông tin dữ liệu:**


<a id="packages"></a>
# **II. Thư viện sử dụng**

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

<a id="collect"></a>
# **III. Thu thập dữ liệu**

In [102]:
BASE_URL = "https://www.nhaccuatui.com/bai-hat/bai-hat-moi.html"

def get_music_types():
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    music_types = []
    menu_browsing = soup.find_all('ul', class_='detail_menu_browsing_dashboard')
    
    for menu in menu_browsing:
        categories = menu.find_all('li', class_='cate')
        for category in categories:
            origin = category.find('a').text.strip()
            subcategories = category.find_next_siblings('li')
            for subcategory in subcategories:
                if subcategory.get('class') and 'line' in subcategory.get('class'):
                    break
                music_type = subcategory.find('a')
                if music_type:
                    title = music_type.text.strip()
                    url = music_type.get('href')
                    music_types.append((title, url, origin))
    
    return music_types

In [106]:
music_types = get_music_types()
pd.DataFrame(music_types, columns=['genre', 'url', 'origin'])

,genre,url,origin
0,Nhạc Trẻ,https://www.nhaccuatui.com/bai-hat/nhac-tre-mo...,VIỆT NAM
1,Trữ Tình,https://www.nhaccuatui.com/bai-hat/tru-tinh-mo...,VIỆT NAM
2,Remix Việt,https://www.nhaccuatui.com/bai-hat/remix-viet-...,VIỆT NAM
3,Rap Việt,https://www.nhaccuatui.com/bai-hat/rap-viet-mo...,VIỆT NAM
4,Tiền Chiến,https://www.nhaccuatui.com/bai-hat/tien-chien-...,VIỆT NAM
5,Nhạc Trịnh,https://www.nhaccuatui.com/bai-hat/nhac-trinh-...,VIỆT NAM
6,Rock Việt,https://www.nhaccuatui.com/bai-hat/rock-viet-m...,VIỆT NAM
7,Cách Mạng,https://www.nhaccuatui.com/bai-hat/cach-mang-m...,VIỆT NAM
8,Pop,https://www.nhaccuatui.com/bai-hat/pop-moi.html,ÂU MỸ
9,Rock,https://www.nhaccuatui.com/bai-hat/rock-moi.html,ÂU MỸ


In [111]:
def get_song_links(music_type_url, pages=25):
    song_links = []
    for page in range(1, pages + 1):
        url = f"{music_type_url[:-5]}.{page}.html"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        for a in soup.select('.list_music_full .info_song a.name_song'):
            song_links.append(a.get('href'))
    
    return song_links

def scrape_song_data(song_url):
    response = requests.get(song_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    name = soup.select_one('.detail_info_playing_now b').text
    artist = soup.select_one('.detail_info_playing_now .name_singer').text if soup.select_one('.detail_info_playing_now .name_singer') else 'N/A'
    # genre = soup.select_one('.detail_info_playing_now b a').text

    composer_element = soup.find('p', string=lambda x: x and 'Nhạc sĩ:' in x)
    composer = composer_element.text.split(': ')[1] if composer_element else 'N/A'
    
    lyrics = soup.select_one('#divLyric').text.strip() if soup.select_one('#divLyric') else 'N/A'
    
    duration = soup.select_one('meta[itemprop="duration"]').get('content')
    default_quality = soup.select_one('#SwitchQualityDefaultflashPlayer').text if soup.select_one('#SwitchQualityDefaultflashPlayer') else 'N/A'
    high_quality = soup.select_one('#SwitchQualityHighflashPlayer').text if soup.select_one('#SwitchQualityHighflashPlayer') else 'N/A'
    
    return {
        'url': song_url,
        'name': name,
        'artist': artist,
        # 'genre': genre,
        'composer': composer,
        'lyrics': lyrics,
        'duration': duration,
        'quality': (default_quality, high_quality)
    }

In [112]:
all_songs = []

for title, url, origin in music_types:
    try:
        song_links = get_song_links(url)
        for song_link in song_links:
            print(song_link)
            song_data = scrape_song_data(song_link)
            song_data['genre'] = title
            song_data['origin'] = origin
        all_songs.append(song_data)
    except Exception as e:
        print(f"Error: {e}")

https://www.nhaccuatui.com/bai-hat/re-ndm-wtran-ft-ca-nau.9gvVeiELZlKq.html
https://www.nhaccuatui.com/bai-hat/yeu-di-chau-khai-phong.W3puBHmNLaTH.html
https://www.nhaccuatui.com/bai-hat/diem-xui-daduc.9IQuJunOhc8Y.html
https://www.nhaccuatui.com/bai-hat/mashup-we-dont-talk-anymore-de-em-roi-xa-bien-cua-hy-vong-duc-phuc.WEMx8xJZYY4g.html
https://www.nhaccuatui.com/bai-hat/giu-em-that-lau-naod.TPDciU83yMeU.html
https://www.nhaccuatui.com/bai-hat/co-duoc-khong-em-cover-xuan-sam.8a0ecQkoye1M.html
https://www.nhaccuatui.com/bai-hat/dung-bat-em-phai-quen-miu-le.y0LT3f6ISOp4.html
https://www.nhaccuatui.com/bai-hat/that-tinh-duc-phuc.QMIetehMFU6R.html
https://www.nhaccuatui.com/bai-hat/gio-lofi-jank-ft-1-9-6-7.cJXTzZkellSD.html
https://www.nhaccuatui.com/bai-hat/ly-do-la-gi-thai-hoc.K6EwXuEHF8Ig.html
https://www.nhaccuatui.com/bai-hat/bui-bay-vao-mat-dong-nhi.lypfICkUx49s.html
https://www.nhaccuatui.com/bai-hat/anh-hua-anh-khong-khoc-khanh-trung-ft-pham-sac-lenh.VXkRUqbWVlQB.html
https://www.

In [114]:
df = pd.DataFrame(all_songs)
df.to_csv('song_data.csv', index=False)